<a href="https://colab.research.google.com/github/castrokelly/Data-Science/blob/main/Abordagem_Ensemble_para_Detec%C3%A7%C3%A3o_e_Classifica%C3%A7%C3%A3o_de_Anomalias_em_Po%C3%A7os_de_Petr%C3%B3leo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abordagem Ensemble para Detecção e Classificação de Anomalias em Poços de Petróleo: um Estudo Aplicado ao Dataset 3W

**Centro de Ciências Matemáticas Aplicadas à Indústria (CeMEAI)**</br>
**Instituto de Ciências Matemáticas e de Computação (ICMC)**</br>
**Universidade de São Paulo**</br>
</br>
Aluna: **Kelly Christine Alvarenga de Castro**
Área de concentração: Ciências de Dados
Orientador: **Prof. Dr. Cláudio Fabiano Motta Toledo**
</br>
---

Este Notebook apresenta o desenvolvimento de um pipeline de aprendizado de máquina para detectar e classificar anomalias em poços de petróleo utilizando o dataset 3W.

A abordagem utiliza um modelo ensemble que, em primeiro lugar, decide se o estado é anômalo (classificação binária: normal vs. anômalo) e, caso seja anômalo, classifica o tipo de evento (rótulos de 1 a 9). O treinamento é realizado com dados de um poço e a validação das métricas é efetuada com dados de outro poço.



Carregamento de dados

In [12]:
!git clone https://github.com/petrobras/3W.git

Cloning into '3W'...
remote: Enumerating objects: 8562, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 8562 (delta 130), reused 115 (delta 114), pack-reused 8369 (from 2)
Receiving objects: 100% (8562/8562), 3.06 GiB | 17.80 MiB/s, done.
Resolving deltas: 100% (2775/2775), done.
Updating files: 100% (2322/2322), done.


In [5]:
!pip install scikeras
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 32.7 MB/s eta 0:00:00


In [7]:
# Importação das Bibliotecas Necessárias
import os
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import glob

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

# Para o modelo LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scikeras.wrappers import KerasClassifier

# Para transformação Wavelet
import pywt
import matplotlib.pyplot as plt
